# Explication des données à manipuler (valeur numérique, échelle  de 0 à 10)
fever – Intensité de la fièvre 

dry_cough – Intensité de la toux sèche

tiredness – Niveau de fatigue

aches_and_pains – Douleurs musculaires ou courbatures

sore_throat – Intensité du mal de gorge

diarrhoea – Sévérité de la diarrhée

conjunctivitis – Intensité de la conjonctivite

headache – Intensité du mal de tête

loss_of_taste – Perte du goût

difficulty_breathing – Difficulté à respirer

chest_pain – Douleur thoracique

rash_on_skin – Présence d’éruptions cutanées

SARS_Cov_2_exam_result – Résultat du test COVID-19 : "positive" ou "negative"

In [1]:
import tenseal as ts
import pandas as pd
import utils
import numpy as np

In [ ]:
covid_data = pd.read_csv('C:/Users/Lambika/Desktop/stage_CETIC/DATA_SET/covid19.csv')

columns = [
    "id","fever", "dry_cough", "tiredness", "aches_and_pains", "sore_throat",
    "diarrhoea", "conjunctivitis", "headache", "loss_of_taste",
    "difficulty_breathing", "chest_pain", "rash_on_skin", "SARS_Cov_2_exam_result"
]

for col in columns:
    globals()[col] = covid_data[col].tolist()
    
# context = ts.context(
#     ts.SCHEME_TYPE.CKKS,
#     poly_modulus_degree=16384,
#     coeff_mod_bit_sizes=[60, 40,40, 60]
# )
# context.global_scale = 2 ** 40 
 
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[40, 21, 21, 21, 21, 21, 21, 40],
)

context.global_scale = 2 ** 21
context.generate_galois_keys()
my_secret_key = context.serialize(save_secret_key=True)
utils.write_data("Keys_provider/secret_key.txt", my_secret_key)
context.make_context_public()
public_key=context.serialize()
utils.write_data("Keys_provider/public_key.txt", public_key)


#cryptage de données
id_encrypted = ts.ckks_vector(context, id)
fever_encrypted = ts.ckks_vector(context, fever)
dry_cough_encrypted = ts.ckks_vector(context, dry_cough)
tiredness_encrypted = ts.ckks_vector(context, tiredness)
aches_and_pains_encrypted = ts.ckks_vector(context, aches_and_pains)
sore_throat_encrypted = ts.ckks_vector(context, sore_throat)
diarrhoea_encrypted = ts.ckks_vector(context, diarrhoea)
conjunctivitis_encrypted = ts.ckks_vector(context, conjunctivitis)
headache_encrypted = ts.ckks_vector(context, headache)
loss_of_taste_encrypted = ts.ckks_vector(context, loss_of_taste)
difficulty_breathing_encrypted = ts.ckks_vector(context, difficulty_breathing)
chest_pain_encrypted = ts.ckks_vector(context, chest_pain)
for i in range(len(SARS_Cov_2_exam_result)):
    if SARS_Cov_2_exam_result[i] == "positive":
        SARS_Cov_2_exam_result[i] = 1
    else:
        SARS_Cov_2_exam_result[i] = 0   
print("sum of covid_positive cases: ", sum(SARS_Cov_2_exam_result))        
rash_on_skin_encrypted = ts.ckks_vector(context, rash_on_skin)

# print(SARS_Cov_2_exam_result)
SARS_Cov_2_exam_result_encrypted = ts.ckks_vector(context, SARS_Cov_2_exam_result)
# Encrypted data serialization
utils.write_data("Provider_Encrypted_data/id_encrypted.txt", id_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/fever_encrypted.txt", fever_encrypted.serialize())  
utils.write_data("Provider_Encrypted_data/dry_cough_encrypted.txt", dry_cough_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/tiredness_encrypted.txt", tiredness_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/aches_and_pains_encrypted.txt", aches_and_pains_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/sore_throat_encrypted.txt", sore_throat_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/diarrhoea_encrypted.txt", diarrhoea_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/conjunctivitis_encrypted.txt", conjunctivitis_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/headache_encrypted.txt", headache_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/loss_of_taste_encrypted.txt", loss_of_taste_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/difficulty_breathing_encrypted.txt", difficulty_breathing_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/chest_pain_encrypted.txt", chest_pain_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/rash_on_skin_encrypted.txt", rash_on_skin_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/SARS_Cov_2_exam_result_encrypted.txt", SARS_Cov_2_exam_result_encrypted.serialize())     


sum of covid_positive cases:  302


In [4]:
from flask import Flask, send_from_directory, render_template, request, redirect, url_for
import os
from werkzeug.utils import secure_filename

app = Flask(__name__)

# Configuration
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'txt'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Créer les dossiers nécessaires
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs('Keys_provider', exist_ok=True)
os.makedirs('Provider_Encrypted_data', exist_ok=True)

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/download/<filename>")
def download_file(filename):
    # Déterminer le bon dossier en fonction du fichier
    if filename == "public_key.txt":
        directory = os.path.join(os.getcwd(), "Keys_provider")
    else:
        directory = os.path.join(os.getcwd(), "Provider_Encrypted_data")
    
    try:
        return send_from_directory(directory, filename, as_attachment=True)
    except FileNotFoundError:
        # Retourner une réponse 404 avec un message personnalisé
        return f"Le fichier {filename} n'a pas été trouvé sur le serveur.", 404

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)
    
    files = request.files.getlist('file')
    
    for file in files:
        if file.filename == '':
            continue
            
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
    
    return redirect(url_for('home'))

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.11.1.153:5000
Press CTRL+C to quit
10.11.1.153 - - [02/Jun/2025 16:02:35] "GET / HTTP/1.1" 200 -
10.11.1.153 - - [02/Jun/2025 16:03:06] "GET /download/id_encrypted.txt HTTP/1.1" 200 -
10.11.1.153 - - [02/Jun/2025 16:04:16] "GET /download/fever_encrypted.txt HTTP/1.1" 200 -
10.11.1.153 - - [02/Jun/2025 16:04:48] "GET /download/SARS_Cov_2_exam_result_encrypted.txt HTTP/1.1" 200 -
10.11.1.153 - - [02/Jun/2025 16:05:07] "GET /download/public_key.txt HTTP/1.1" 200 -
10.11.1.153 - - [02/Jun/2025 16:05:15] "GET /download/chest_pain_encrypted.txt HTTP/1.1" 200 -
10.11.1.153 - - [02/Jun/2025 16:05:23] "GET /download/dry_cough_encrypted.txt HTTP/1.1" 200 -
10.11.1.153 - - [02/Jun/2025 16:08:17] "POST /upload HTTP/1.1" 302 -
10.11.1.153 - - [02/Jun/2025 16:08:17] "GET / HTTP/1.1" 200 -
10.11.1.153 - - [02/Jun/2025 16:08:23] "POST /upload HTTP/1.1" 302 -
10.11.1.153 - - [02/Jun/2025 16:08:23] "GET / H

In [ ]:
loaded_secret_key = utils.read_data("Keys_provider/secret_key.txt")
secret_key = ts.context_from(loaded_secret_key).secret_key()

nombre_de_cas_positive = utils.read_data("uploads/SARS_Cov_2_exam_result_SARS_Cov_2_exam_result_encrypted_computed.txt")
nombre_de_cas_positive = ts.lazy_ckks_vector_from(nombre_de_cas_positive)
nombre_de_cas_positive.link_context(context)
sigmoid_enc = utils.read_data("uploads/fever_sigmoided_manipulated_encrypted.txt")
sigmoid_vector = ts.lazy_ckks_vector_from(sigmoid_enc)
sigmoid_vector.link_context(context)

##dechiffrage
decrypted = nombre_de_cas_positive.decrypt(secret_key)
result = np.round(decrypted[0], decimals=0) 
sigmoid_decrypted = sigmoid_vector.decrypt(secret_key)
print(f"Résultat brut (avec erreur CKKS): {decrypted[0]}")
print(f"Résultat corrigé: {int(result)}") 
print("")
print("------------------------------------")
print("")
score = utils.read_data("uploads/score_added_encrypted.txt")
score = ts.lazy_ckks_vector_from(score)
score.link_context(context)
decrypted_score_addition =score.decrypt(secret_key)
print(f"Score brut version addition (avec erreur CKKS): {decrypted_score_addition}")
score_addition = [a + b + c for a, b, c in zip(chest_pain, dry_cough, fever)]

print("real score brut de l'addition", score_addition)
fièvre_detectée = [1 if val > 0.5 else 0 for val in sigmoid_decrypted]
presence_fievre = [1 if val > 8 else 0 for val in fever]


print("")
print("------------------------------------")
print("")
score = utils.read_data("uploads/score_multiplication_encrypted.txt")
score = ts.lazy_ckks_vector_from(score)
score.link_context(context)
decrypted_score_mult =score.decrypt(secret_key)

print(f"Score brut version multiplication (avec erreur CKKS): {decrypted_score_mult}")
score_mult = [a * b * c for a, b, c in zip(chest_pain, dry_cough, fever)]
print("real score brut de multiplication", score_mult)

print("")
print("------------------------------------")
print("")
print("Fièvre détectée ?(real)",presence_fievre)
print("Fièvre détectée ?(avec erreur CKKS)", fièvre_detectée)

diff_count = sum(1 for a, b in zip(fièvre_detectée, presence_fievre) if a != b)
print(f"nombre de différence avec les deux listes : {diff_count}")

Résultat brut (avec erreur CKKS): 303.00482049204174
Résultat corrigé: 303

------------------------------------

Score brut version addition (avec erreur CKKS): [18.103446275008228, 22.362650871304478, 24.247403998993466, 13.345174566278509, 13.263875688296608, 15.985475163802676, 19.893067039215804, 17.699892318029143, 12.729761654395585, 8.168287069414522, 16.351852172596438, 15.7869648549731, 22.62702608041513, 9.232834294596557, 17.470818545470987, 12.47354196207183, 11.094274312548645, 18.67856736354662, 19.43659553229826, 18.243956500300705, 10.181366817201367, 8.101394459363151, 26.132173735252927, 8.486074648994276, 15.238900765889577, 19.81225144286914, 11.093728614190669, 19.811414810384875, 11.09288200042219, 11.475147181763901, 7.550760418366743, 13.317478662521255, 23.610259133883915, 10.980869973933107, 16.96743593284976, 23.55221574578517, 17.268232316052597, 17.62112910297266, 9.80389655400954, 15.403768803504185, 9.052124035048138, 23.64815087500537, 22.07094467208312